In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# -------------------------------
# PARAMÈTRES
# -------------------------------
ENEDIS_FILE = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/ban_enedis.csv"
ADEME_FILE  = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/dpe_sample_2M_clean.csv"

# -------------------------------
# LOAD FILES
# -------------------------------
df_enedis = pd.read_csv(ENEDIS_FILE)
df_ademe  = pd.read_csv(ADEME_FILE)

# -------------------------------
# PRINT HEADERS
# -------------------------------
print("===== HEADERS ENEDIS =====")
print(list(df_enedis.columns))

print("\n===== HEADERS ADEME =====")
print(list(df_ademe.columns))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
===== HEADERS ENEDIS =====
['adress_ban', 'ban_lat', 'ban_lon', 'ban_score', 'ban_id', 'adresse_clean']

===== HEADERS ADEME =====
['conso_5_usages_ef', 'code_insee_ban', 'deperditions_planchers_bas', 'numero_voie_ban', 'conso_5_usages_ep', 'date_fin_validite_dpe', 'adresse_brut', 'conso_5_usages_par_m2_ep', 'date_etablissement_dpe', 'conso_5_usages_par_m2_ef', 'nom_commune_ban', 'annee_construction', 'date_visite_diagnostiqueur', 'nom_rue_ban', 'qualite_isolation_murs', 'type_batiment', 'periode_construction', 'type_installation_ecs', 'numero_dpe', 'qualite_isolation_menuiseries', 'date_reception_dpe', 'adresse_ban', 'date_derniere_modification_dpe', 'etiquette_ges', 'qualite_isolation_enveloppe', 'type_installation_chauffage', 'code_postal_ban', 'surface_habitable_logement', 'etiquette_dpe', '_score', 'adresse_source', 'adresse_clean', 'conso_theorique_loge

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# -------------------------------
# PARAMÈTRES
# -------------------------------
ENEDIS_FILE = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/ban_enedis.csv"
ADEME_FILE  = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/dpe_sample_2M_clean.csv"

# -------------------------------
# LOAD
# -------------------------------
df_enedis = pd.read_csv(ENEDIS_FILE)
df_ademe  = pd.read_csv(ADEME_FILE)

# normaliser les colonnes
df_enedis.columns = df_enedis.columns.str.lower()
df_ademe.columns  = df_ademe.columns.str.lower()

# -------------------------------
# EXTRAIRE LES LISTES UNIQUES
# -------------------------------
enedis_add = set(df_enedis["adress_ban"].dropna().astype(str))
ademe_add  = set(df_ademe["adresse_ban"].dropna().astype(str))

# -------------------------------
# INTERSECTION
# -------------------------------
intersection = enedis_add & ademe_add

print("Nombre d'adresses communes :", len(intersection))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📌 Nombre d'adresses communes : 83304


# On essaye d'augmenter l'intersection

In [ ]:
import pandas as pd
import unicodedata
from google.colab import drive

drive.mount('/content/drive')

# -------------------------------
# FICHIERS
# -------------------------------
ADEME_FILE  = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/raw/dpe_sample_2M.csv"
ENEDIS_FILE = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/ban_enedis.csv"
OUTPUT      = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/raw/dpe_sample_2M_ban_corrected.csv"

# -------------------------------
# LOAD
# -------------------------------
df_ademe = pd.read_csv(ADEME_FILE)
df_enedis = pd.read_csv(ENEDIS_FILE)

df_ademe.columns = df_ademe.columns.str.lower()
df_enedis.columns = df_enedis.columns.str.lower()

# -------------------------------
# UTILS
# -------------------------------
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def clean(s):
    s = str(s).lower().strip()
    s = strip_accents(s)
    s = s.replace(",", " ").replace(".", " ")
    s = " ".join(s.split())
    return s

def is_empty(s):
    """Vraies valeurs vides : '', None, nan, ., , etc."""
    if s is None:
        return True
    s = str(s).strip().lower()
    return s in ["", "nan", "none", ".", ","]

# Nettoyage ENEDIS
df_enedis = df_enedis[df_enedis["adress_ban"].notna()]
enedis_raw     = df_enedis["adress_ban"].astype(str).tolist()
enedis_cleaned = df_enedis["adress_ban"].astype(str).apply(clean).tolist()
map_enedis = dict(zip(enedis_cleaned, enedis_raw))

# -------------------------------
# CLEAN COMMUNE
# -------------------------------
def clean_commune(com):
    com = str(com).strip()
    if "(" in com:
        com = com.split("(")[0].strip()
    return com

# -------------------------------
# RECONSTRUCT ADEME
# -------------------------------
def reconstruct_ademe(row):
    adr = str(row.get("adresse_brut", "")).strip()
    cp  = str(row.get("code_postal_brut", "")).strip()
    com = clean_commune(row.get("nom_commune_brut", ""))

    # Correction CP style xxxxx.0
    if cp.replace(".", "").isdigit():
        try:
            cp = str(int(float(cp))).zfill(5)
        except:
            pass

    parts = [adr, cp, com]
    parts = [p for p in parts if not is_empty(p)]
    return " ".join(parts)

# -------------------------------
# MATCH ADEME -> ENEDIS
# -------------------------------
def update_ademe_ban(row):

    ban_avant = str(row.get("adresse_ban", "")).strip()

    # ❌ Cas 1 — adresse_ban déjà valide et non vide
    if not is_empty(ban_avant):
        return ban_avant

    # reconstruction ADEME
    adr_brut  = row.get("adresse_brut", "")
    ademe_reconstructed = reconstruct_ademe(row)
    ademe_clean = clean(ademe_reconstructed)

    # ❌ Cas 2 — trop court → on arrête (évite faux positifs)
    if len(ademe_clean.split()) < 3:
        return ademe_reconstructed

    # numéro de voie ADEME
    num_voie_ademe = str(row.get("numero_voie_ban", "")).strip()
    num_voie_ademe_clean = ''.join(filter(str.isdigit, num_voie_ademe))

    # matching
    for enedis_c in enedis_cleaned:

        # Si ADEME est contenu dans ENEDIS
        if ademe_clean in enedis_c:

            # ⚠ Vérif du numéro de voie (si dispo)
            num_enedis = ''.join(filter(str.isdigit, enedis_c.split(" ")[0]))
            if num_voie_ademe_clean and num_voie_ademe_clean != num_enedis:
                continue  # → faux match, on skip

            enedis_orig = map_enedis[enedis_c]

            print("\n[MATCH TROUVÉ]")
            print("ADEME brute :", adr_brut)
            print("ADEME recon :", ademe_reconstructed)
            print("→ ENEDIS :", enedis_orig)

            return enedis_orig

    # aucun match → garder ADEME
    return ademe_reconstructed

# -------------------------------
# APPLY
# -------------------------------
df_ademe["adresse_ban"] = df_ademe.apply(update_ademe_ban, axis=1)

# -------------------------------
# COLONNES UTILES
# -------------------------------
colonnes_utiles = [
    "numero_dpe",
    "adresse_ban", "adresse_brut",
    "numero_voie_ban", "nom_rue_ban", "nom_commune_ban",
    "code_postal_ban", "code_insee_ban",
    "surface_habitable_logement",
    "etiquette_dpe", "etiquette_ges",
    "conso_5_usages_par_m2_ep", "conso_5_usages_ep",
    "conso_5_usages_par_m2_ef", "conso_5_usages_ef",
    "type_batiment",
    "periode_construction", "annee_construction",
    "date_etablissement_dpe", "date_fin_validite_dpe",
    "date_reception_dpe", "date_visite_diagnostiqueur",
    "date_derniere_modification_dpe",
    "qualite_isolation_enveloppe",
    "qualite_isolation_murs",
    "qualite_isolation_plancher_bas",
    "qualite_isolation_menuiseries",
    "type_installation_chauffage",
    "type_installation_ecs"
]

colonnes_existantes = [c for c in colonnes_utiles if c in df_ademe.columns]
df_out = df_ademe[colonnes_existantes]

df_out.to_csv(OUTPUT, index=False)

print("\n✔ Correction terminée")
print("📁 Fichier sauvegardé :", OUTPUT)
print("Colonnes gardées :", colonnes_existantes)


Streaming output truncated to the last 5000 lines.
[MATCH TROUVÉ]
ADEME brute : 13 PLACE GRANVILLE,
ADEME recon : 13 PLACE GRANVILLE,
→ ENEDIS : 13 Place Granville 21000 Dijon

[MATCH TROUVÉ]
ADEME brute : 82 RUE DE LA FORET,
ADEME recon : 82 RUE DE LA FORET,
→ ENEDIS : 82 Rue de la Forêt 27000 Évreux

[MATCH TROUVÉ]
ADEME brute : 14 ALLEE DU PARC,
ADEME recon : 14 ALLEE DU PARC,
→ ENEDIS : 14 Allée du Parc 71200 Le Creusot

[MATCH TROUVÉ]
ADEME brute : 313 AVENUE ANDREI SAKHAROV,
ADEME recon : 313 AVENUE ANDREI SAKHAROV,
→ ENEDIS : 313 Avenue Andrei Sakharov 69009 Lyon

[MATCH TROUVÉ]
ADEME brute : 26 rue des Alouettes,
ADEME recon : 26 rue des Alouettes,
→ ENEDIS : 26 Rue des Alouettes 69008 Lyon

[MATCH TROUVÉ]
ADEME brute : 13 RUE DES COQUELICOTS,
ADEME recon : 13 RUE DES COQUELICOTS,
→ ENEDIS : 13 Rue des Coquelicots 25550 Bavans

[MATCH TROUVÉ]
ADEME brute : 103 AVENUE DE PARIS,
ADEME recon : 103 AVENUE DE PARIS,
→ ENEDIS : 103 Avenue de Paris 92320 Châtillon

[MATCH TROUVÉ]
ADEM

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# -------------------------------
# PARAMÈTRES
# -------------------------------
ENEDIS_FILE = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/ban_enedis.csv"
ADEME_FILE  = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/raw/dpe_sample_2M_ban_corrected.csv"

# -------------------------------
# LOAD
# -------------------------------
df_enedis = pd.read_csv(ENEDIS_FILE)
df_ademe  = pd.read_csv(ADEME_FILE)

# normaliser les colonnes
df_enedis.columns = df_enedis.columns.str.lower()
df_ademe.columns  = df_ademe.columns.str.lower()

# -------------------------------
# EXTRAIRE LES LISTES UNIQUES
# -------------------------------
enedis_add = set(df_enedis["adress_ban"].dropna().astype(str))
ademe_add  = set(df_ademe["adresse_ban"].dropna().astype(str))

# -------------------------------
# INTERSECTION
# -------------------------------
intersection = enedis_add & ademe_add

print("📌 Nombre d'adresses communes :", len(intersection))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📌 Nombre d'adresses communes : 85845


In [ ]:
ADEME_FILE  = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/raw/dpe_sample_2M_ban_corrected.csv"

df_ademe  = pd.read_csv(ADEME_FILE)

df_ademe[df_ademe["adresse_ban"].str.contains("1 Allée Abbé Desgranges",case=False, na=False)]

,numero_dpe,adresse_ban,adresse_brut,numero_voie_ban,nom_rue_ban,nom_commune_ban,code_postal_ban,code_insee_ban,surface_habitable_logement,etiquette_dpe,...,date_etablissement_dpe,date_fin_validite_dpe,date_reception_dpe,date_visite_diagnostiqueur,date_derniere_modification_dpe,qualite_isolation_enveloppe,qualite_isolation_murs,qualite_isolation_menuiseries,type_installation_chauffage,type_installation_ecs
223490,2187E0216838H,1 allée Abbé Desgranges 87100 Limoges,1 Allée Abbé Desgranges,NaN,1 Allée Abbé Desgranges,LIMOGES,87100.0,87085,68.0,F,...,2021-08-16,2031-08-15,2021-08-17,2021-08-16,2024-02-06,insuffisante,insuffisante,moyenne,collectif,collectif
233791,2187E0216842L,1 allée Abbé Desgranges 87100 Limoges,1 Allée Abbé Desgranges,NaN,1 Allée Abbé Desgranges,LIMOGES,87100.0,87085,72.0,D,...,2021-08-22,2031-08-21,2021-08-23,2021-08-22,2024-02-06,bonne,insuffisante,moyenne,collectif,collectif


In [ ]:
import pandas as pd
import unicodedata
from google.colab import drive

drive.mount('/content/drive')

# ----------------------------
# FICHIERS
# ----------------------------
ENEDIS_FILE  = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/ban_enedis.csv"
ADEME_V1     = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/raw/dpe_sample_2M.csv"               # ancienne version
ADEME_V2     = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/raw/dpe_sample_2M_ban_corrected.csv" # nouvelle version

OUTPUT_ADDED   = "/content/drive/MyDrive/Hackathon_2025/datasets/clean/intersection_added.csv"
OUTPUT_REMOVED = "/content/drive/MyDrive/Hackathon_2025/datasets/clean/intersection_removed.csv"

# ----------------------------
# CLEANING UTILS
# ----------------------------
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def clean(s):
    s = str(s).lower().strip()
    s = strip_accents(s)
    s = s.replace(",", " ")
    s = " ".join(s.split())
    return s

# ----------------------------
# LOAD
# ----------------------------
df_enedis = pd.read_csv(ENEDIS_FILE)
df_v1     = pd.read_csv(ADEME_V1)
df_v2     = pd.read_csv(ADEME_V2)

df_enedis.columns = df_enedis.columns.str.lower()
df_v1.columns     = df_v1.columns.str.lower()
df_v2.columns     = df_v2.columns.str.lower()

# ----------------------------
# ADRESSES UNIQUES + CLEAN
# ----------------------------
enedis_set = set(df_enedis["adress_ban"].dropna().astype(str).apply(clean).unique())

v1_set = set(df_v1["adresse_ban"].dropna().astype(str).apply(clean).unique())
v2_set = set(df_v2["adresse_ban"].dropna().astype(str).apply(clean).unique())

# intersections
intersection_v1 = enedis_set & v1_set
intersection_v2 = enedis_set & v2_set

print("\n===== INTERSECTION STATS =====")
print(f"V1 intersection : {len(intersection_v1):,}")
print(f"V2 intersection : {len(intersection_v2):,}")

# ----------------------------
# DIFFERENCES
# ----------------------------
added   = sorted(list(intersection_v2 - intersection_v1))  # ce que V2 a gagné
removed = sorted(list(intersection_v1 - intersection_v2))  # ce que V2 a perdu

print("\n===== CHANGEMENTS =====")
print(f"✔ Ajoutés dans V2 (nouvelle version) : {len(added):,}")
print(f"✔ Retirés dans V2 : {len(removed):,}")

print("\nExemples ajoutés (10) :", added[:10])
print("Exemples retirés (10) :", removed[:10])

# ----------------------------
# SAVE RESULTS
# ----------------------------
pd.DataFrame({"adresse": added}).to_csv(OUTPUT_ADDED, index=False)
pd.DataFrame({"adresse": removed}).to_csv(OUTPUT_REMOVED, index=False)

print("\n✔ Fichiers enregistrés :")
print("Ajouts   :", OUTPUT_ADDED)
print("Retraits :", OUTPUT_REMOVED)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import unicodedata

# ----------------------------
# FICHIERS (adapte les chemins)
# ----------------------------
ADEME_FILE  = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/dpe_sample_2M_clean.csv"
ENEDIS_FILE = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/ban_enedis.csv"
ADEME_V2     = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/raw/dpe_sample_2M_ban_corrected.csv"
INTER_OUT    = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/intersection_adresses_enedis_ademe.csv"

# ----------------------------
# FONCTIONS DE CLEAN
# ----------------------------
def strip_accents(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', str(s))
        if unicodedata.category(c) != 'Mn'
    )

def clean(s):
    s = str(s).lower().strip()
    s = strip_accents(s)
    s = s.replace(",", " ")
    s = " ".join(s.split())
    return s

# ----------------------------
# LOAD
# ----------------------------
df_enedis = pd.read_csv(ENEDIS_FILE)
df_v2     = pd.read_csv(ADEME_V2)

df_enedis.columns = df_enedis.columns.str.lower()
df_v2.columns     = df_v2.columns.str.lower()

# Sécurité : vérifier que les colonnes existent
assert "adress_ban"  in df_enedis.columns, "Colonne 'adress_ban' absente dans ENEDIS"
assert "adresse_ban" in df_v2.columns,     "Colonne 'adresse_ban' absente dans ADEME V2"

# ----------------------------
# SETS D'ADRESSES CLEAN
# ----------------------------
enedis_clean = set(
    df_enedis["adress_ban"].dropna().astype(str).apply(clean).unique()
)
v2_clean = set(
    df_v2["adresse_ban"].dropna().astype(str).apply(clean).unique()
)

# ----------------------------
# INTERSECTION = ce qui est commun ENEDIS ∩ ADEME V2
# ----------------------------
intersection_v2 = sorted(list(enedis_clean & v2_clean))

print(f"📌 Nombre d'adresses communes (V2) : {len(intersection_v2)}")

# ----------------------------
# SAUVEGARDE DANS LE FICHIER DEMANDÉ
# ----------------------------
df_inter = pd.DataFrame({"adresse": intersection_v2})
df_inter.to_csv(INTER_OUT, index=False)

print("✔ Fichier d'intersection sauvegardé sous :", INTER_OUT)


📌 Nombre d'adresses communes (V2) : 91261
✔ Fichier d'intersection sauvegardé sous : /content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/intersection_adresses_enedis_ademe.csv


In [ ]:
import pandas as pd
import unicodedata
from google.colab import drive

drive.mount('/content/drive')

# -------------------------------
# FICHIERS
# -------------------------------
ADEME_FILE  = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/raw/dpe_sample_2M_ban_corrected.csv"
ENEDIS_FILE = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/ban_enedis.csv"
INTER_FILE  = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/intersection_adresses_enedis_ademe.csv"

OUT_ENEDIS = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/enedis_ban_final.csv"
OUT_ADEME  = "/content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/ademe_final.csv"

# -------------------------------
# LOAD
# -------------------------------
df_ademe = pd.read_csv(ADEME_FILE)
df_enedis = pd.read_csv(ENEDIS_FILE)
df_inter = pd.read_csv(INTER_FILE)

df_ademe.columns = df_ademe.columns.str.lower()
df_enedis.columns = df_enedis.columns.str.lower()

# -------------------------------
# CLEAN FUNCTION
# -------------------------------
import unicodedata

def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def clean(s):
    s = str(s).lower().strip()
    s = strip_accents(s)
    s = s.replace(",", " ")
    s = " ".join(s.split())
    return s

# -------------------------------
# PREP
# -------------------------------
intersection = df_inter["adresse"].astype(str).apply(clean).tolist()

df_enedis["adress_ban_clean"] = df_enedis["adress_ban"].astype(str).apply(clean)
df_ademe["adresse_ban_clean"] = df_ademe["adresse_ban"].astype(str).apply(clean)

# -------------------------------
# 1️⃣ ENEDIS — UNIQUES
# -------------------------------
df_enedis_unique = df_enedis[df_enedis["adress_ban_clean"].isin(intersection)]
df_enedis_unique = df_enedis_unique.drop_duplicates(subset=["adress_ban_clean"])

df_enedis_unique.to_csv(OUT_ENEDIS, index=False)
print("✔ ENEDIS uniques sauvegardés :", OUT_ENEDIS, "|", len(df_enedis_unique))

# -------------------------------
# 2️⃣ ADEME — UNIQUES
# -------------------------------
df_ademe_unique = df_ademe[df_ademe["adresse_ban_clean"].isin(intersection)]
df_ademe_unique = df_ademe_unique.drop_duplicates(subset=["adresse_ban_clean"])

df_ademe_unique.to_csv(OUT_ADEME, index=False)
print("✔ ADEME uniques sauvegardés  :", OUT_ADEME, "|", len(df_ademe_unique))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✔ ENEDIS uniques sauvegardés : /content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/enedis_ban_final.csv | 91261
✔ ADEME uniques sauvegardés  : /content/drive/MyDrive/Colab_Notebooks/Hackathon_2025/datasets/clean/ademe_final.csv | 91261
